In [2]:
! pip install datasets transformers seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=cbd584922e1c98fc19a1623a4479b9ed6f15d460c7cfa4a25f2ccaa2865ffbf3
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
import transformers

print(transformers.__version__)

4.44.2


In [6]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

# Fine-tuning a model on a token classification task

In [7]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-uncased"
batch_size = 16

## Loading the dataset

In [8]:
from datasets import load_dataset

Using the [CONLL 2003 dataset](https://www.aclweb.org/anthology/W03-0419.pdf).

In [9]:
datasets = load_dataset("conll2003")

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [11]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [12]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [13]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [14]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [15]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,4138,"[Diana, 's, office, said, on, Saturday, the, princess, had, sent, a, message, to, the, Nobel, Peace, Prize-winning, missionary, as, news, broke, this, week, of, her, battle, against, heart, problems, and, malaria, .]","[NNP, POS, NN, VBD, IN, NNP, DT, NNS, VBD, VBN, DT, NN, TO, DT, NNP, NNP, JJ, JJ, IN, NN, VBD, DT, NN, IN, PRP$, NN, IN, NN, NNS, CC, NN, .]","[B-NP, B-NP, I-NP, B-VP, B-PP, B-NP, B-NP, I-NP, B-VP, I-VP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, B-PP, B-NP, B-VP, B-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, O, B-NP, O]","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MISC, I-MISC, I-MISC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,7863,"[AMSTERDAM, 1996-08-26]","[NN, CD]","[B-NP, I-NP]","[B-LOC, O]"
2,4876,"[Aoki, ,, the, elder, brother, of, reigning, 125cc, world, champion, Haruchika, ,, had, a, race-long, duel, with, John, Kocinski, of, the, United, States, on, a, Ducati, before, taking, the, chequered, flag, .]","[NNP, ,, DT, JJR, NN, IN, VBG, CD, NN, NN, NNP, ,, VBD, DT, JJ, NN, IN, NNP, NNP, IN, DT, NNP, NNP, IN, DT, NNP, IN, VBG, DT, VBN, NN, .]","[B-NP, O, B-NP, I-NP, I-NP, B-PP, B-VP, B-NP, I-NP, I-NP, B-NP, O, B-VP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-VP, B-NP, I-NP, I-NP, O]","[B-PER, O, O, O, O, O, O, O, O, O, B-PER, O, O, O, O, O, O, B-PER, I-PER, O, O, B-LOC, I-LOC, O, O, B-ORG, O, O, O, O, O, O]"
3,19,"[It, brought, in, 4,275, tonnes, of, British, mutton, ,, some, 10, percent, of, overall, imports, .]","[PRP, VBD, IN, CD, NNS, IN, JJ, NN, ,, DT, CD, NN, IN, JJ, NNS, .]","[B-NP, B-VP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, O, B-NP, I-NP, I-NP, B-PP, B-NP, I-NP, O]","[O, O, O, O, O, O, B-MISC, O, O, O, O, O, O, O, O, O]"
4,2033,"[6-3, 6-4]","[CD, CD]","[B-NP, I-NP]","[O, O]"
5,1965,"[Extras, (, lb-12, w-1, nb-10, ), 23]","[NNS, (, CD, JJ, JJ, ), CD]","[B-NP, O, B-NP, I-NP, I-NP, O, B-NP]","[O, O, O, O, O, O, O]"
6,7663,"[CHICAGO, 1996-08-26]","[NNP, CD]","[B-NP, I-NP]","[B-LOC, O]"
7,13038,"[PHILADELPHIA, 1996-08-29]","[NNP, CD]","[B-NP, I-NP]","[B-LOC, O]"
8,4132,"[When, asked, if, he, would, return, to, the, mountainous, region, where, rebels, have, been, fighting, Russian, troops, for, full, independence, ,, Penrose, said, :, "", Not, for, the, time, being, .]","[WRB, VBN, IN, PRP, MD, VB, TO, DT, JJ, NN, WRB, NNS, VBP, VBN, VBG, JJ, NNS, IN, JJ, NN, ,, NNP, VBD, :, "", RB, IN, DT, NN, VBG, .]","[B-ADVP, B-VP, B-SBAR, B-NP, B-VP, I-VP, B-PP, B-NP, I-NP, I-NP, B-ADVP, B-NP, B-VP, I-VP, I-VP, B-NP, I-NP, B-PP, B-NP, I-NP, O, B-NP, B-VP, O, O, O, B-PP, B-NP, I-NP, B-VP, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-MISC, O, O, O, O, O, B-PER, O, O, O, O, O, O, O, O, O]"
9,13893,"[--, Julia, Ferguson, ,, Vienna, newsroom, ,, +431, 53112, 274]","[:, NNP, NNP, ,, NNP, NN, ,, CD, CD, CD]","[B-NP, I-NP, I-NP, O, B-NP, I-NP, O, B-NP, I-NP, I-NP]","[O, B-PER, I-PER, O, B-LOC, O, O, O, O, O]"


## Preprocessing the data

In [17]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [19]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [21]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [22]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


In [23]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 39)

In [24]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [25]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


In [26]:
label_all_tokens = True

In [27]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [28]:
tokenize_and_align_labels(datasets['train'][:2])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100]]}

In [29]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

## Fine-tuning the model

In [30]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [41]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True, 
    report_to="none"
)

In [32]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [33]:
!pip install evaluate

import evaluate

metric = evaluate.load("seqeval")

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00


In [34]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [35]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [42]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [43]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.074266,0.891060,0.913189,0.901989,0.979252
2,0.193600,0.059805,0.923051,0.936682,0.929817,0.984098
3,0.050700,0.057408,0.929888,0.942164,0.935986,0.985035


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=1317, training_loss=0.10112733341295246, metrics={'train_runtime': 401.9999, 'train_samples_per_second': 104.784, 'train_steps_per_second': 3.276, 'total_flos': 1129700749077486.0, 'train_loss': 0.10112733341295246, 'epoch': 3.0})

In [44]:
trainer.evaluate()

{'eval_loss': 0.0574079230427742,
 'eval_precision': 0.9298884840454896,
 'eval_recall': 0.9421635529701309,
 'eval_f1': 0.9359857746165815,
 'eval_accuracy': 0.985035029469236,
 'eval_runtime': 10.4395,
 'eval_samples_per_second': 311.317,
 'eval_steps_per_second': 9.771,
 'epoch': 3.0}

In [47]:
trainer.predict(tokenized_datasets["test"]).metrics

{'test_loss': 0.12342683970928192,
 'test_precision': 0.884174848059841,
 'test_recall': 0.8978162829337764,
 'test_f1': 0.890943351784242,
 'test_accuracy': 0.9749671983979007,
 'test_runtime': 10.086,
 'test_samples_per_second': 342.354,
 'test_steps_per_second': 10.708}

In [48]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'LOC': {'precision': 0.9557321225879682,
  'recall': 0.9648586707410237,
  'f1': 0.960273712222011,
  'number': 2618},
 'MISC': {'precision': 0.8250197941409343,
  'recall': 0.8464662875710804,
  'f1': 0.8356054530874099,
  'number': 1231},
 'ORG': {'precision': 0.899856938483548,
  'recall': 0.9178015564202334,
  'f1': 0.9087406693956177,
  'number': 2056},
 'PER': {'precision': 0.9715127701375246,
  'recall': 0.9779169413315755,
  'f1': 0.9747043363994744,
  'number': 3034},
 'overall_precision': 0.9298884840454896,
 'overall_recall': 0.9421635529701309,
 'overall_f1': 0.9359857746165815,
 'overall_accuracy': 0.985035029469236}

You can now upload the result of the training to the Hub, just execute this instruction:

In [49]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/AmrataYadav/bert-base-uncased-finetuned-ner/commit/ab2f64f92a3af5bd40f8265d51fc1d30435d0599', commit_message='End of training', commit_description='', oid='ab2f64f92a3af5bd40f8265d51fc1d30435d0599', pr_url=None, pr_revision=None, pr_num=None)